In [3]:
%load_ext autoreload
%autoreload 2

from models import RNN
from utils import DataManager
import tensorflow as tf
from tensorpack import (TrainConfig, TestDataSpeed, SyncMultiGPUTrainerParameterServer, 
                        SaverRestore, logger)
from tensorpack.callbacks import (ScalarPrinter, ScheduledHyperParamSetter, GPUUtilizationTracker)

image_table_loc = "./data/standard_images.csv"
annot_table_loc = "./data/standard_annotations.csv"
resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
logger.set_logger_dir('train_log')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1009 16:54:39 @logger.py:93] WRN Log directory train_log exists! Please either backup/delete it, or use a new directory.
[1009 16:54:39 @logger.py:95] WRN If you're resuming from a previous run you can choose to keep it.
[1009 16:54:39 @logger.py:96] Select Action: k (keep) / b (backup) / d (delete) / n (new) / q (quit):
d
[1009 16:54:41 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-4f2afd69-65c1-4739-b613-f943e7dad4e4.json


In [2]:
data_manager = DataManager(image_table_loc, annot_table_loc)
train_data = data_manager.get_validation_stream()

In [10]:
tf.reset_default_graph()
ignore_restore = ['process', 'logits', 'learning_rate', 'beta1_power', 'beta2_power']
model = RNN(read_time=5, label_num=40, batch_size=32, weight_decay=0.0)
train_config = TrainConfig(model=model, dataflow=train_data, 
                           callbacks=[ScheduledHyperParamSetter('learning_rate', [(0, 1e-4)]),
                                      GPUUtilizationTracker()], 
                           session_init=SaverRestore(model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=10, nr_tower=2)

[1009 17:18:36 @prof.py:38] WRN [GPUUtilizationTracker] Both devices and CUDA_VISIBLE_DEVICES are None! Will monitor all visible GPUs!
[1009 17:18:36 @gpu.py:32] Loading devices by TensorFlow ...


In [11]:
SyncMultiGPUTrainerParameterServer(train_config).train()

[1009 17:18:38 @input_source.py:235] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1009 17:18:38 @input_source.py:509] Setting up StagingArea for GPU prefetching ...
[1009 17:18:38 @multigpu.py:66] Training a model of 2 towers
[1009 17:18:38 @multigpu.py:87] Building graph for training tower 0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype

KeyboardInterrupt: 

In [8]:
train_data.reset_state()
train_data.size()

4

In [26]:
tf.get_collection(tf.GraphKeys.)

[]